In [2]:
%matplotlib notebook
%load_ext autoreload
%autoreload 1
!hostname
!pwd

dv002.bridges2.psc.edu
/ocean/projects/asc170022p/mtragoza/mre-pinn/MICCAI-2023


In [3]:
import sys, os, tqdm
import numpy as np
import xarray as xr
import torch

os.environ['DDEBACKEND'] = 'pytorch'
import deepxde

sys.path.append('..')
%aimport mre_pinn

sys.path.append('../../mre_ai')
#import mre_ai.mre_datasets

torch.cuda.is_available()

True

## Process images for a single patient

- load SITK images from NIFTI files
- perform filtering/inpainting on wave image
- register images wrt anatomical image
- load segmentation model
- perform image segmentation
- convert images to xarray Dataset

In [4]:
%autoreload
patient = mre_pinn.data.ImagingPatient('0006')
patient.load_images()
patient.metadata

Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/t1_pre_in.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/t1_pre_water.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/t1_pre_out.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/t1_pre_fat.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/t2.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/mre_raw.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/wave.nii
Loading /ocean/projects/asc170022p/shared/Data/MRE/MRE_DICOM_7-31-19/NIFTI/0006/mre.nii


size    spacing      origin       limit      center  \
sequence     dimension                                                        
t1_pre_in    0           512   0.703100 -204.130005  155.154108  -24.487948   
             1           512   0.703100 -208.591995  150.692118  -28.949939   
             2            88   2.300003 -142.013000   58.087265  -41.962868   
t1_pre_water 0           512   0.703100 -204.130005  155.154108  -24.487948   
             1           512   0.703100 -208.591995  150.692118  -28.949939   
             2            88   2.300003 -142.013000   58.087265  -41.962868   
t1_pre_out   0           512   0.703100 -204.130005  155.154108  -24.487948   
             1           512   0.703100 -208.591995  150.692118  -28.949939   
             2            88   2.300003 -142.013000   58.087265  -41.962868   
t1_pre_fat   0           512   0.703100 -204.130005  155.154108  -24.487948   
             1           512   0.703100 -208.591995  150.692118  -28.949939   
             2            88   2.300003 -142.013000   58.087265  -41.962868   
t2           0           256   1.406300 -200.263000  158.343486  -20.959757   
             1           256   1.406300 -208.240997  150.365490  -28.937754   
             2            23  10.000000 -154.516006   65.483994  -44.516006   
mre_raw      0           256   1.406300 -206.904007  151.702480  -27.600763   
             1           256   1.406300 -195.858994  162.747493  -16.555750   
             2             4   1.000000    0.000000    3.000000    1.500000   
wave         0           512   1.000000    0.000000  511.000000  255.500000   
             1           512   1.000000    0.000000  511.000000  255.500000   
             2             4   1.000000    0.000000    3.000000    1.500000   
mre          0           256   1.406300 -206.904007  151.702480  -27.600763   
             1           256   1.406300 -195.858994  162.747493  -16.555750   
             2             4  11.000000   -1.056400   31.943600   15.443600   

                            extent  
sequence     dimension              
t1_pre_in    0          359.987213  
             1          359.987213  
             2          202.400269  
t1_pre_water 0          359.987213  
             1          359.987213  
             2          202.400269  
t1_pre_out   0          359.987213  
             1          359.987213  
             2          202.400269  
t1_pre_fat   0          359.987213  
             1          359.987213  
             2          202.400269  
t2           0          360.012787  
             1          360.012787  
             2          230.000000  
mre_raw      0          360.012787  
             1          360.012787  
             2            4.000000  
wave         0          512.000000  
             1          512.000000  
             2            4.000000  
mre          0          360.012787  
             1          360.012787  
             2           44.000000

In [6]:
%autoreload
patient.preprocess(same_grid=True, mre_size=(256, 256, 4), anat_size=(256, 256, 4))

Correcting metadata on mre_raw
Correcting metadata on wave
Restoring wave
Registering t1_pre_out to mre_raw
ParameterObject (0x55905def4f30)
  RTTI typeinfo:   elastix::ParameterObject
  Reference Count: 1
  Modified Time: 36498
  Debug: Off
  Object Name: 
  Observers: 
    none
ParameterMap 0: 
  (CenterOfRotationPoint -27.6008 -16.5558 15.4436)
  (CompressResultImage "false")
  (ComputeZYX "false")
  (DefaultPixelValue 0)
  (Direction 1 0 0 0 1 0 0 0 1)
  (FinalBSplineInterpolationOrder 3)
  (FixedImageDimension 3)
  (FixedInternalImagePixelType "float")
  (HowToCombineTransforms "Compose")
  (Index 0 0 0)
  (InitialTransformParametersFileName "NoInitialTransform")
  (MovingImageDimension 3)
  (MovingInternalImagePixelType "float")
  (NumberOfParameters 6)
  (Origin -207.243 -196.198 -84.6065)
  (ResampleInterpolator "FinalBSplineInterpolator")
  (Resampler "DefaultResampler")
  (ResultImageFormat "nii")
  (ResultImagePixelType "float")
  (Size 512 512 88)
  (Spacing 0.7031 0.7031 2

  (UseDirectionCosines "true")
Resizing t1_pre_in to (256, 256, 4)
Resizing t1_pre_water to (256, 256, 4)
Resizing t1_pre_out to (256, 256, 4)
Resizing t1_pre_fat to (256, 256, 4)
Resizing t2 to (256, 256, 4)
Resizing mask to (256, 256, 4)
Resizing mre_raw to (256, 256, 4)
Resizing wave to (256, 256, 4)
Resizing mre to (256, 256, 4)
Resizing mask to (256, 256, 4)
Correcting metadata on t1_pre_in
Correcting metadata on t1_pre_water
Correcting metadata on t1_pre_out
Correcting metadata on t1_pre_fat
Correcting metadata on t2
Correcting metadata on mask
Correcting metadata on mre_raw
Correcting metadata on wave
Correcting metadata on mre
Correcting metadata on mask


In [7]:
patient.metadata

size  spacing      origin       limit     center  \
sequence     dimension                                                     
t1_pre_in    0           256   1.4063 -206.904007  151.702480 -27.600763   
             1           256   1.4063 -195.858994  162.747493 -16.555750   
             2             4  11.0000   -1.056400   31.943600  15.443600   
t1_pre_water 0           256   1.4063 -206.904007  151.702480 -27.600763   
             1           256   1.4063 -195.858994  162.747493 -16.555750   
             2             4  11.0000   -1.056400   31.943600  15.443600   
t1_pre_out   0           256   1.4063 -206.904007  151.702480 -27.600763   
             1           256   1.4063 -195.858994  162.747493 -16.555750   
             2             4  11.0000   -1.056400   31.943600  15.443600   
t1_pre_fat   0           256   1.4063 -206.904007  151.702480 -27.600763   
             1           256   1.4063 -195.858994  162.747493 -16.555750   
             2             4  11.0000   -1.056400   31.943600  15.443600   
t2           0           256   1.4063 -206.904007  151.702480 -27.600763   
             1           256   1.4063 -195.858994  162.747493 -16.555750   
             2             4  11.0000   -1.056400   31.943600  15.443600   
mre_raw      0           256   1.4063 -206.904007  151.702480 -27.600763   
             1           256   1.4063 -195.858994  162.747493 -16.555750   
             2             4  11.0000   -1.056400   31.943600  15.443600   
wave         0           256   1.4063 -206.904007  151.702480 -27.600763   
             1           256   1.4063 -195.858994  162.747493 -16.555750   
             2             4  11.0000   -1.056400   31.943600  15.443600   
mre          0           256   1.4063 -206.904007  151.702480 -27.600763   
             1           256   1.4063 -195.858994  162.747493 -16.555750   
             2             4  11.0000   -1.056400   31.943600  15.443600   
anat_mask    0           256   1.4063 -206.904007  151.702480 -27.600763   
             1           256   1.4063 -195.858994  162.747493 -16.555750   
             2             4  11.0000   -1.056400   31.943600  15.443600   
mre_mask     0           256   1.4063 -206.904007  151.702480 -27.600763   
             1           256   1.4063 -195.858994  162.747493 -16.555750   
             2             4  11.0000   -1.056400   31.943600  15.443600   

                            extent  
sequence     dimension              
t1_pre_in    0          360.012787  
             1          360.012787  
             2           44.000000  
t1_pre_water 0          360.012787  
             1          360.012787  
             2           44.000000  
t1_pre_out   0          360.012787  
             1          360.012787  
             2           44.000000  
t1_pre_fat   0          360.012787  
             1          360.012787  
             2           44.000000  
t2           0          360.012787  
             1          360.012787  
             2           44.000000  
mre_raw      0          360.012787  
             1          360.012787  
             2           44.000000  
wave         0          360.012787  
             1          360.012787  
             2           44.000000  
mre          0          360.012787  
             1          360.012787  
             2           44.000000  
anat_mask    0          360.012787  
             1          360.012787  
             2           44.000000  
mre_mask     0          360.012787  
             1          360.012787  
             2           44.000000

In [15]:
%autoreload
example = mre_pinn.data.MREExample.from_patient(patient)
example.anat

Converting t1_pre_in to xarray
Converting t1_pre_water to xarray
Converting t1_pre_out to xarray
Converting t1_pre_fat to xarray
Converting t2 to xarray
Converting mre_raw to xarray
Converting wave to xarray
Converting mre to xarray
Converting mask to xarray
Converting mask to xarray


<xarray.DataArray 't1_pre_in' (x: 256, y: 256, z: 4, component: 5)>
array([[[[ 3.91698034e-20, -4.19973473e-10,  2.30173620e-20,
          -5.22206455e-20, -7.72612035e-26],
         [-1.41440001e-19, -3.83624965e-10, -3.12337228e-20,
          -4.68830462e-20, -6.98464841e-26],
         [ 5.52396908e-21, -3.28948008e-10, -8.09280424e-20,
          -1.19850354e-19,  9.00861743e-27],
         [-2.39978699e-20, -2.44738230e-10, -3.27558009e-21,
           1.33647933e-19,  7.41968170e-28]],

        [[ 5.19202142e-19,  2.25905669e-10,  3.06531034e-19,
          -7.13152679e-19,  2.95008242e-25],
         [-1.96586421e-18,  2.49976456e-10, -4.59558439e-19,
          -6.74966610e-19,  2.72294940e-25],
         [ 1.11043285e-19,  2.37245112e-10, -1.09133412e-18,
          -1.66069402e-18, -3.54719102e-26],
         [-3.58104220e-19,  1.02129423e-10, -6.64715167e-20,
           1.86872591e-18, -2.92639720e-27]],

        [[ 6.90299716e-18, -9.30554189e-10,  4.08365585e-18,
          -9.73233164e-18, -1.12221213e-24],
...
         [ 0.00000000e+00,  2.88539313e-19,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  7.81875349e-20,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00, -5.60750261e-21,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00, -1.73219731e-20,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00, -7.27389737e-20,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  8.79544700e-21,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00, -7.95772740e-22,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00, -1.77745422e-21,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00, -6.71476423e-21,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00]]]], dtype=float32)
Coordinates:
  * x          (x) float64 -0.4138 -0.411 -0.4082 ... 0.2978 0.3006 0.3034
  * y          (y) float64 -0.3917 -0.3889 -0.3861 ... 0.3199 0.3227 0.3255
  * z          (z) float64 -0.002113 0.01989 0.04189 0.06389
  * component  (component) <U12 't1_pre_in' 't1_pre_out' ... 't1_pre_fat' 't2'
    region     (x, y, z) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [16]:
example.wave

<xarray.DataArray 'wave' (x: 256, y: 256, z: 4)>
array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
...
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 2.59313541e-25,  9.06511984e-21,  3.10481994e-26,
          7.32113561e-27],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]]])
Coordinates:
  * x        (x) float64 -0.4138 -0.411 -0.4082 -0.4054 ... 0.2978 0.3006 0.3034
  * y        (y) float64 -0.3917 -0.3889 -0.3861 ... 0.3199 0.3227 0.3255
  * z        (z) float64 -0.002113 0.01989 0.04189 0.06389
    region   (x, y, z) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [17]:
example.save_xarrays('../data/NAFLD/test/')

Writing ../data/NAFLD/test/0006/wave.nc
Writing ../data/NAFLD/test/0006/mre.nc
Writing ../data/NAFLD/test/0006/mre_mask.nc
Writing ../data/NAFLD/test/0006/anat.nc
Writing ../data/NAFLD/test/0006/anat_mask.nc


In [20]:
%autoreload
example = mre_pinn.data.MREExample.load_xarrays('../data/NAFLD/test/', '0006', anat=True)
example.anat

Loading ../data/NAFLD/test/0006/wave.nc
Loading ../data/NAFLD/test/0006/mre.nc
Loading ../data/NAFLD/test/0006/mre_mask.nc
Loading ../data/NAFLD/test/0006/anat.nc
Loading ../data/NAFLD/test/0006/anat_mask.nc


<xarray.DataArray 't1_pre_in' (x: 256, y: 256, z: 4, component: 5)>
[1310720 values with dtype=float32]
Coordinates:
  * x          (x) float64 -0.4138 -0.411 -0.4082 ... 0.2978 0.3006 0.3034
  * y          (y) float64 -0.3917 -0.3889 -0.3861 ... 0.3199 0.3227 0.3255
  * z          (z) float64 -0.002113 0.01989 0.04189 0.06389
  * component  (component) object 't1_pre_in' 't1_pre_out' ... 't1_pre_fat' 't2'
    region     (x, y, z) float32 ...

## Process images for a patient cohort

In [ ]:
with open('/ocean/projects/asc170022p/shared/Data/MRE/gold.txt') as f:
    patient_ids = [line.split('.')[0].split('_')[1] for line in f]

len(patient_ids)

In [ ]:
%autoreload
cohort = mre_pinn.data.ImagingCohort(patient_ids='{' + ','.join(patient_ids) + '}')
len(cohort)

In [ ]:
%%time
cohort.load_images()

In [ ]:
%%time
cohort.preprocess()

In [ ]:
%%time
dataset = mre_pinn.data.MREDataset.from_cohort(cohort)

In [ ]:
dataset.save_xarrays('../data/NAFLD/v3')

In [ ]:
%autoreload
dataset[0].view('wave', 'mre', 'anat', mask=True)